In [1]:
import numpy as np
import pandas as pd

In [2]:
cc_data = pd.read_csv('data/cleaned_confirmed_cases.csv')

In [3]:
cc_data['DayOfYear'] =  pd.to_datetime(cc_data['Date']).dt.dayofyear

In [4]:
cc_data = pd.concat([cc_data,pd.get_dummies(cc_data['Country_Region'], prefix='country')],axis=1)

In [5]:
cc_data.head()

,Country_Region,Population,Weight,Date,TargetValue,Med. Age,Density (P/Km²),Urban Pop %,Week,Day,...,country_United States,country_Uruguay,country_Uzbekistan,country_Venezuela,country_Vietnam,country_West Bank and Gaza,country_Western Sahara,country_Yemen,country_Zambia,country_Zimbabwe
0,Afghanistan,27657145,0.058359,2020-01-23,0.0,18.0,60,25.0,4,23,...,0,0,0,0,0,0,0,0,0,0
1,Afghanistan,27657145,0.058359,2020-01-24,0.0,18.0,60,25.0,4,24,...,0,0,0,0,0,0,0,0,0,0
2,Afghanistan,27657145,0.058359,2020-01-25,0.0,18.0,60,25.0,4,25,...,0,0,0,0,0,0,0,0,0,0
3,Afghanistan,27657145,0.058359,2020-01-26,0.0,18.0,60,25.0,4,26,...,0,0,0,0,0,0,0,0,0,0
4,Afghanistan,27657145,0.058359,2020-01-27,0.0,18.0,60,25.0,5,27,...,0,0,0,0,0,0,0,0,0,0


In [6]:
cc_data.drop(['Country_Region', 'first_infection'],axis=1, inplace=True)

In [7]:
feature_cols = list(cc_data)
feature_cols.remove('TargetValue')
feature_cols.remove('Date')

In [8]:
# cc_data.sort_values('Day')

In [9]:
mask_cc_test = (cc_data['Date'] >= '2020-05-20')
mask_cc_train = (cc_data['Date'] < '2020-05-20')

In [10]:
test_cc_df = cc_data.loc[mask_cc_test]
train_cc_df = cc_data.loc[mask_cc_train]

In [11]:
y_train_cc = train_cc_df['TargetValue']
x_train_cc = train_cc_df[feature_cols] 

y_test_cc = test_cc_df['TargetValue']
x_test_cc = test_cc_df[feature_cols]

In [12]:
from sklearn import linear_model

In [13]:
reg = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))

In [14]:
reg.fit(x_train_cc, y_train_cc)

RidgeCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
        cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
        scoring=None, store_cv_values=False)

In [15]:
reg.score(x_test_cc, y_test_cc)

-404026.41260942887

In [16]:
from sklearn.linear_model import LinearRegression

In [17]:
lr = LinearRegression().fit(x_train_cc, y_train_cc)

In [18]:
lr.score(x_test_cc, y_test_cc)

0.4757987734841689

In [19]:
from sklearn.linear_model import SGDRegressor

In [20]:
from sklearn.pipeline import make_pipeline

In [21]:
from sklearn.preprocessing import StandardScaler

In [22]:
sgd = make_pipeline(StandardScaler(),SGDRegressor(max_iter=1000, tol=1e-3))

In [23]:
sgd.fit(x_train_cc, y_train_cc)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('sgdregressor',
                 SGDRegressor(alpha=0.0001, average=False, early_stopping=False,
                              epsilon=0.1, eta0=0.01, fit_intercept=True,
                              l1_ratio=0.15, learning_rate='invscaling',
                              loss='squared_loss', max_iter=1000,
                              n_iter_no_change=5, penalty='l2', power_t=0.25,
                              random_state=None, shuffle=True, tol=0.001,
                              validation_fraction=0.1, verbose=0,
                              warm_start=False))],
         verbose=False)

In [24]:
sgd.score(x_test_cc, y_test_cc)

0.4575656086339108

In [25]:
lasso = linear_model.Lasso(alpha=0.1)

In [26]:
lasso.fit(x_train_cc, y_train_cc)

/Users/larry/miniconda3/envs/mldm/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:474: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14492623110.435894, tolerance: 5265598.493604564
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [27]:
lasso.score(x_test_cc, y_test_cc)

0.4738739713569108

In [28]:
from sklearn.linear_model import ElasticNet

In [30]:
regr = ElasticNet(random_state=0)

In [31]:
regr.fit(x_train_cc, y_train_cc)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=0, selection='cyclic', tol=0.0001, warm_start=False)

In [32]:
regr.score(x_test_cc, y_test_cc)

0.07925488507885159

In [33]:
from sklearn.ensemble import GradientBoostingRegressor

In [34]:
gbr = GradientBoostingRegressor(random_state=0)

In [35]:
gbr.fit(x_train_cc, y_train_cc)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=0, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [36]:
gbr.score(x_test_cc, y_test_cc)

0.7875191730845746

In [37]:
from sklearn.experimental import enable_hist_gradient_boosting

In [38]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [39]:
est = HistGradientBoostingRegressor().fit(x_train_cc, y_train_cc)

In [40]:
est.score(x_test_cc, y_test_cc)

0.8625675564712547

In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

In [42]:
regressor = DecisionTreeRegressor(random_state=0)

In [43]:
cross_val_score(regressor, x_train_cc, y_train_cc, cv=10)

array([ 0.16490867,  0.05459928, -4.89102934,  0.20466636, -1.24204917,
       -0.14833218,  0.15117956,  0.10672793, -0.95623815,  0.04322886])

In [44]:
regressor.scor(x_test_cc, y_test_cc)

NotFittedError: This DecisionTreeRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.